In [ ]:
import os
import json
import requests
import threading
import time
import datetime
import pandas as pd
from getpass import getpass
from pprint import pprint

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# User inputs
TILE_NUMBER = 'h3v9'
DATASET_NAME = 'landsat_ard_tile_c2'
SENSORS = [
    'LT05',  # Only processing LT05 for now
]
CLOUD_COVER_FILTER = {'min': 0, 'max': 75}
FILE_TYPE = 'band'
DATA_DIR = 'data'
MAX_THREADS = 5

# Load AOI information
aoi_df = pd.read_csv('tile_aoi.csv')
aoi_df = aoi_df[aoi_df['tile'] == TILE_NUMBER].iloc[0]
spatial_filter = {
    "filterType": "mbr",
    "lowerLeft": {"latitude": aoi_df['lly'], "longitude": aoi_df['llx']},
    "upperRight": {"latitude": aoi_df['ury'], "longitude": aoi_df['urx']}
}

pprint(spatial_filter)

# Helper function to create directories
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)
        print(f"Directory '{path}' created successfully.")
    else:
        print(f"Directory '{path}' already exists.")

# HTTP request sender
def send_request(url, payload, api_key=None):
    headers = {'X-Auth-Token': api_key} if api_key else {}
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    result = response.json()
    if result.get('errorCode'):
        raise Exception(f"{result['errorCode']}: {result['errorMessage']}")
    return result['data']

# Function to remove previous scene lists
def remove_scene_list(service_url, list_id, api_key):
    try:
        payload = {"listId": list_id}
        send_request(f"{service_url}scene-list-remove", payload, api_key)
        print(f"Successfully removed list: {list_id}")
    except Exception as e:
        print(f"Failed to remove list {list_id}: {e}")

# Download function with retry
def download_file(url):
    sema.acquire()
    try:
        response = requests.get(url, stream=True)
        filename = response.headers['content-disposition'].split('filename=')[-1].strip("\"")
        print(f"Downloading: {filename}...")
        with open(os.path.join(DATA_DIR, filename), 'wb') as file:
            file.write(response.content)
    except Exception as e:
        print(f"Failed to download from {url}. Retrying...")
        run_download(url)
    finally:
        sema.release()

# Threaded download
def run_download(url):
    thread = threading.Thread(target=download_file, args=(url,))
    threads.append(thread)
    thread.start()

# Login to USGS M2M API
def login_to_usgs(service_url):
    username = input("Enter USGS Username: ")
    token = getpass("Enter USGS API Token: ")
    response = requests.post(f"{service_url}login-token", json={"username": username, "token": token})
    response.raise_for_status()
    return response.json()['data']

# Main execution
def main():
    service_url = "https://m2m.cr.usgs.gov/api/api/json/stable/"
    api_key = login_to_usgs(service_url)

    create_directory(DATA_DIR)

    for sensor in SENSORS:
        print(f"Processing sensor: {sensor}")
        temporal_ranges = {
            'LT05': {'start': '2006-01-01', 'end': '2012-05-05'},
        }
        temporal_coverage = temporal_ranges[sensor]
        date_ranges = pd.date_range(start=temporal_coverage['start'], end=temporal_coverage['end'], freq='YS')
        date_ranges = date_ranges.append(pd.to_datetime([temporal_coverage['end']]))

        for i in range(len(date_ranges) - 1):
            temporal_filter = {'start': str(date_ranges[i].date()), 'end': str(date_ranges[i + 1].date())}
            payload = {
                'datasetName': DATASET_NAME,
                'sceneFilter': {
                    'spatialFilter': spatial_filter,
                    'acquisitionFilter': temporal_filter,
                    'cloudCoverFilter': CLOUD_COVER_FILTER
                }
            }
            scenes = send_request(f"{service_url}scene-search", payload, api_key)

            # Debugging logs
            print(f"Payload for {sensor}: {payload}")
            pprint(scenes)

            entity_ids = [s['entityId'] for s in scenes['results'] if s['options']['bulk']]
            print(f"Entity IDs for {sensor}: {entity_ids}")

            if not entity_ids:
                continue

            list_id = f"{sensor}_scene_list"
            remove_scene_list(service_url, list_id, api_key)  # Clean up previous lists
            list_payload = {"listId": list_id, "idField": "entityId", "entityIds": entity_ids, "datasetName": DATASET_NAME}
            send_request(f"{service_url}scene-list-add", list_payload, api_key)

            download_payload = {"listId": list_id, "datasetName": DATASET_NAME}
            products = send_request(f"{service_url}download-options", download_payload, api_key)
            downloads = [{"entityId": p["entityId"], "productId": p["id"]} for p in products if p['bulkAvailable']]
            download_request = {"downloads": downloads, "label": datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}
            download_results = send_request(f"{service_url}download-request", download_request, api_key)

            for dl in download_results['availableDownloads']:
                run_download(dl['url'])

if __name__ == "__main__":
    sema = threading.Semaphore(MAX_THREADS)
    threads = []
    main()
